# GEST-Schätzung für Waldgebiete

Für Waldgebiete kann eine vereinfachte Schätzung der CO2-Äq-Emissionen vor und nach den Maßnahmen erfolgen.

Skizze: 

lösung in qgis, abgleich wasserstufen mit biotoptypliste von dbu (spalte in attributtabelle). 

## Schritt 1:
Vergleichbar mit dem Skript für die Offenland-Flächen wird als PyQGIS Code ein neues .shp File mit den Waldbiotopen der zu untersuchenden Fläche erzeugt.

**Das folgende Skript bitte in der QGIS Python-Konsole ausführen!**

In [ ]:
# HIER bitte die Namen der Layer im GIS angeben 
biotope_name = "Biotope_GEST"
torfboeden_name = "Torfboeden"
output_name = "output"

# Am Ende des Skripts ist der Name für die Output-Datei 
# angegeben (aktuell offenlandbiotope.shp). 
# Bei Bedarf ändern. 

import sys
import os 

from qgis.core import (
    QgsProject, QgsVectorLayer, QgsFeatureRequest,
    QgsVectorFileWriter, QgsProcessingFeatureSourceDefinition
)

biotope_layer = QgsProject.instance().mapLayersByName(biotope_name)[0]
torfboeden_layer = QgsProject.instance().mapLayersByName(torfboeden_name)[0]

# HIER Spalten- und Featurename ändern
expression = '"Hckurz" IN (\'Wälder\')'

# In den folgenden auskommentierten Zeilen kann (falls gewünscht) überprüft werden, wie viele Biotope extrahiert werden würden. 

# features = biotope_layer.getFeatures(QgsFeatureRequest().setFilterExpression(expression))
# count = sum(1 for f in features)
# print(f"Gefundene Features: {count}")

biotope_filtered = biotope_layer.materialize(QgsFeatureRequest().setFilterExpression(expression))

params = {
    'INPUT': biotope_filtered,
    'PREDICATE': [0],  # "intersects"
    'INTERSECT': torfboeden_layer,
    'METHOD': 0,       
    'OUTPUT': 'memory:'
}
result = processing.run("native:extractbylocation", params)
extracted_layer = result['OUTPUT']

torfboeden_path = torfboeden_layer.dataProvider().dataSourceUri().split("|")[0]
output_dir = os.path.dirname(torfboeden_path)

# HIER Name der Outputdatei ändern
output_path = os.path.join(output_dir, output_name)

QgsVectorFileWriter.writeAsVectorFormat(
    extracted_layer,
    output_path,
    "UTF-8",
    driverName="ESRI Shapefile"
)

print(f"Shapefile gespeichert unter: {output_path}")

## Schritt 2: CO2-Äq-Emissionen vor Maßnahmenumsetzung in GIS bestimmen. 


Zuerst muss manuell in QGIS die bereits erstellte Flurabstandsplankarte neu kategorisiert (nach KOSKA) und anschließend nach Tabelle reklassifiziert werden.

Die Biotoptypbezeichnung sowie Wasserstufe nach Koska soll mit der GEST_Wald_Mastertabelle verglichen und darauf basierend in eine neue Spalte "Ist_Emissionen" der Wert der CO2-Äq-Emissionen in t/ha/a angegeben werden. Außerdem soll ein Dictionary nicht_gefunden = {} erstellt werden, wo die Kombinationen aus Biotop und Wasserstufe gespeichert werden, die nicht in der Mastertabelle erhalten sind. Diese müssen dann manuell eingetragen werden.  

**Vor Anwendung die Pfade ändern und Namen der Layer anpassen (bei vector_layer und raster_layer)!**


In [ ]:
import pandas as pd
import openpyxl
from qgis.PyQt.QtCore import QVariant
from qgis.core import QgsRaster, QgsProject, QgsRasterLayer, QgsFeatureRequest, QgsCoordinateTransform, QgsWkbTypes

# HIER die richtigen Pfade angeben
shapefile_path = r"...waldbiotope.shp" # Waldbiotope
raster_path = r"..." # TIF Raster mit KOSKA Wasserstufen des Gebiets 
excel_path = r"...xlsx" # GEST Wald Mastertabelle mit den Wasserstufen aller Arten

# Wie heißen die Layer im GIS? 
raster_layer = QgsRasterLayer(raster_path, "KOSKA_SchSee_2014") # ANPASSEN
vector_layer = QgsVectorLayer(shapefile_path, "waldbiotope", "ogr") # ANPASSEN

if not vector_layer.isValid() or not raster_layer.isValid():
    raise Exception("Layer laden nicht möglich.")

# mastertabelle laden
df = pd.read_excel(excel_path, engine="openpyxl")
master_dict = {(str(row[0]).strip(), str(row[1]).strip()): row[2] for row in df.values}

# neue spalte in der attributtabelle der waldbiotope anlegen
if "Ist_Emi" not in [f.name() for f in vector_layer.fields()]:
    vector_layer.dataProvider().addAttributes([QgsField("Ist_Emi", QVariant.Double)])
    vector_layer.updateFields()

if "KOSKA" not in [f.name() for f in vector_layer.fields()]:
    vector_layer.dataProvider().addAttributes([QgsField("KOSKA", QVariant.String)])
    vector_layer.updateFields()

nicht_gefunden = {}

crs_vector = vector_layer.crs()
crs_raster = raster_layer.crs()
transform = QgsCoordinateTransform(crs_vector, crs_raster, QgsProject.instance())

raster_provider = raster_layer.dataProvider()
raster_extent = raster_layer.extent()
band = 1  

with edit(vector_layer):
    # print([f.name() for f in vector_layer.fields()])
    for feature in vector_layer.getFeatures():
        geom = feature.geometry()
        if QgsWkbTypes.geometryType(geom.wkbType()) != QgsWkbTypes.PolygonGeometry:
            continue  

        centroid = geom.centroid().asPoint()
        transformed_point = transform.transform(centroid)

        # Rasterwert abfragen
        ident = raster_provider.identify(transformed_point, QgsRaster.IdentifyFormatValue)
        if not ident.isValid():
            continue

        raster_value = ident.results().get(band)
        if raster_value is None:
            continue

        try:
            raster_str = str(int(raster_value))  # falls float wie 3.0 → "3"
        except:
            raster_str = str(raster_value)

        lbtname = str(feature["LBTNAME"]).strip()

        key = (lbtname, raster_str)

        if key in master_dict:
            feature["Ist_Emi"] = master_dict[key]
            feature["KOSKA"] = raster_str
        else:
            feature["Ist_Emi"] = None
            feature["KOSKA"] = raster_str
            nicht_gefunden[key] = True 

        vector_layer.updateFeature(feature)

print("Nicht gefundene Kombinationen (bitte manuell prüfen):")
for k in nicht_gefunden:
    print(f"  {k}")


## Schritt 3: CO2-Äq-Emissionen nach Maßnahme bestimmen 

1. Im GIS Soll-Flurabstandskarten mithilfe des Prognose-Tools erzeugen
2. Diese Karten reklassifizieren nach KOSKA
3. Gleichen Code wie in Schritt 2 für die neuen Karten anwenden, nur heißt Ist_Emi jetzt Soll_Emi (unten rauskopieren und Pfade anpassen)

In [ ]:
import openpyxl
import pandas as pd
from qgis.PyQt.QtCore import QVariant
from qgis.core import QgsRaster, QgsProject, QgsRasterLayer, QgsFeatureRequest, QgsCoordinateTransform, QgsWkbTypes

# HIER die richtigen Pfade angeben
shapefile_path = r"...shp" # Waldbiotope
raster_path = r"...tif" # TIF Raster mit KOSKA Wasserstufen des Gebiets 
excel_path = r"...xlsx" # GEST Wald Mastertabelle mit den Wasserstufen aller Arten


# Wie heißen die Layer im GIS? 
raster_layer = QgsRasterLayer(raster_path, "KOSKA_SchSee_2014") # ANPASSEN
vector_layer = QgsVectorLayer(shapefile_path, "waldbiotope", "ogr") # ANPASSEN

if not vector_layer.isValid() or not raster_layer.isValid():
    raise Exception("Layer laden nicht möglich.")

# mastertabelle laden
df = pd.read_excel(excel_path, engine="openpyxl")
master_dict = {(str(row[0]).strip(), str(row[1]).strip()): row[2] for row in df.values}

# neue spalte in der attributtabelle der waldbiotope anlegen
if "Soll_Emi" not in [f.name() for f in vector_layer.fields()]:
    vector_layer.dataProvider().addAttributes([QgsField("Soll_Emi", QVariant.Double)])
    vector_layer.updateFields()

if "KOSKA" not in [f.name() for f in vector_layer.fields()]:
    vector_layer.dataProvider().addAttributes([QgsField("KOSKA", QVariant.String)])
    vector_layer.updateFields()

nicht_gefunden = {}

crs_vector = vector_layer.crs()
crs_raster = raster_layer.crs()
transform = QgsCoordinateTransform(crs_vector, crs_raster, QgsProject.instance())

raster_provider = raster_layer.dataProvider()
raster_extent = raster_layer.extent()
band = 1  

with edit(vector_layer):
    # print([f.name() for f in vector_layer.fields()])
    for feature in vector_layer.getFeatures():
        geom = feature.geometry()
        if QgsWkbTypes.geometryType(geom.wkbType()) != QgsWkbTypes.PolygonGeometry:
            continue  

        centroid = geom.centroid().asPoint()
        transformed_point = transform.transform(centroid)

        # Rasterwert abfragen
        ident = raster_provider.identify(transformed_point, QgsRaster.IdentifyFormatValue)
        if not ident.isValid():
            continue

        raster_value = ident.results().get(band)
        if raster_value is None:
            continue

        try:
            raster_str = str(int(raster_value))  
        except:
            raster_str = str(raster_value)

        lbtname = str(feature["LBTNAME"]).strip()

        key = (lbtname, raster_str)

        if key in master_dict:
            feature["Soll_Emi"] = master_dict[key]
            feature["KOSKA"] = raster_str
        else:
            feature["Soll_Emi"] = None
            feature["KOSKA"] = raster_str
            nicht_gefunden[key] = True 

        vector_layer.updateFeature(feature)

print("Nicht gefundene Kombinationen (bitte manuell prüfen):")
for k in nicht_gefunden:
    print(f"  {k}")
